In [2]:
import argparse
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, ConcatDataset

import os
import json
import resource
import sys
import pickle

sys.path.insert(1, 'src')
from model import Model
from utils import *
from data import *
from train_baseline import *

In [3]:
parser = create_arg_parser()
tgt_market = 't1'
src_markets = 's1-s2'

args = parser.parse_args(f'--tgt_market {tgt_market} --src_markets {src_markets}'.split()) #

if torch.cuda.is_available() and args.cuda:
    torch.cuda.set_device(0)
args.device = torch.device('cuda' if torch.cuda.is_available() and args.cuda else 'cpu')

In [4]:
# load pretrained model
model_dir = 'checkpoints/t1_s1-s2_toytest.model'
id_bank_dir = 'checkpoints/t1_s1-s2_toytest.pickle'

valid_run = 'valid_t1_s1-s2_toytest.tsv'

with open(id_bank_dir, 'rb') as centralid_file:
    my_id_bank = pickle.load(centralid_file)

mymodel = Model(args, my_id_bank)
mymodel.load(model_dir)

Model is GMF++!
GMF(
  (embedding_user): Embedding(16273, 8)
  (embedding_item): Embedding(11074, 8)
  (affine_output): Linear(in_features=8, out_features=1, bias=True)
  (logistic): Sigmoid()
)
Pretrained weights from checkpoints/t1_s1-s2_toytest.model are loaded!


In [5]:
############
## Target Market data
############
tgt_train_data_dir = os.path.join(args.data_dir, args.tgt_market, 'train.tsv')
tgt_train_ratings = pd.read_csv(tgt_train_data_dir, sep='\t')

print(f'loading {tgt_train_data_dir}')
tgt_task_generator = TaskGenerator(tgt_train_ratings, my_id_bank)
print('loaded target data!')

tgt_valid_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_valid, args.batch_size)
tgt_test_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_test, args.batch_size)

loading DATA/t1/train.tsv
loaded target data!


In [ ]:
valid_run_mf = mymodel.predict(tgt_valid_dataloader)

In [ ]:
tgt_valid_qrel = read_qrel_file('DATA/t1/valid_qrel.tsv')
task_ov, task_ind = get_evaluations_final(valid_run_mf, tgt_valid_qrel)

In [ ]:
print(task_ov)